### Making call to data generator to get the .csv files


In [14]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
import subprocess
from sklearn.model_selection import train_test_split
from pathlib import Path
import os

In [15]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd() / ".." / ".."))


In [16]:
from source.sp_data_generator.supply_planning_data_generator import SupplyPlanningDataGenerator

In [ ]:
generator = SupplyPlanningDataGenerator(seed=42)

# Generate a complete dataset
dataset = generator.generate_full_dataset(
    num_products=20,
    num_facilities=5,
    periods=52,
    frequency='W',  # Weekly data
    start_date='2023-01-01'
)

for key in dataset.keys():
    key_str = str(key) + ".csv"
    df = dataset[key]
    file_path = os.path.join("data", key_str)
    df.to_csv(file_path, index=True)




In [ ]:
#Only get first 4 products to test model
df = pd.read_csv("data/demand.csv")
prod_list = ['P0001', 'P0002', 'P0003', 'P0004']
demand_abbreviated_df = df[df['product_id'].isin(prod_list)]


In [ ]:
# Adding a month feature column taken from the data column, encoded as a number 1-12, also encouding the facilitys 1 through 3
demand_abbreviated_df["day_of_year"] = pd.to_datetime(demand_abbreviated_df['date']).dt.dayofyear
demand_abbreviated_df["Month"] = demand_abbreviated_df['date'].str[5:7].astype(int)
demand_abbreviated_df["facility_id"] = demand_abbreviated_df["facility_id"].str[2:].astype(int)
demand_abbreviated_df.drop(columns=['date'], inplace=True)



In [ ]:
demand_abbreviated_df["demand"].describe()
demand_abbreviated_df.head(150)

In [ ]:
result_item1 = demand_abbreviated_df[demand_abbreviated_df['product_id'] == 'P0001']
df_month_sum1 = result_item1.groupby(['Month']).sum()['demand']

result_item2 = demand_abbreviated_df[demand_abbreviated_df['product_id'] == 'P0002']
df_month_sum2 = result_item2.groupby(['Month']).sum()['demand']

result_item3 = demand_abbreviated_df[demand_abbreviated_df['product_id'] == 'P0003']
df_month_sum3 = result_item3.groupby(['Month']).sum()['demand']

result_item4 = demand_abbreviated_df[demand_abbreviated_df['product_id'] == 'P0004']
df_month_sum4 = result_item4.groupby(['Month']).sum()['demand']

fig, ax = plt.subplots(1,1)
df_month_sum1.plot(x='Month', y='demand', label='P0001')
df_month_sum2.plot(x='Month', y='demand', label='P0002')
df_month_sum3.plot(x='Month', y='demand', label='P0003')
df_month_sum4.plot(x='Month', y='demand', label='P0004')

ax.set_xlabel('Month')
ax.set_ylabel('Demand')
ax.set_title('Average Monthly Demand of 4 Products')
ax.legend()
plt.show()

In [ ]:
demand_total = demand_abbreviated_df['demand']
plt.hist(demand_total, bins=50)
plt.title('Histogram of Demand')
plt.show()

stats.probplot(demand_total, dist='norm', plot=plt)
plt.title('Normal Q-Q Plot of Demand')
plt.show()

print(stats.shapiro(demand_total))

In [ ]:
demand_abbreviated_df['product_id'] = demand_abbreviated_df['product_id'].str[3:].astype(int)

In [ ]:
from sklearn.metrics import mean_squared_error

X = demand_abbreviated_df.drop(columns=['demand'])
y = demand_abbreviated_df['demand']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")